# Module 2. Using AWS Lambda Layers
---


## Lambda Layer

In [ ]:
PROJECT_LAYER = "layerapp"
!rm -rf $PROJECT_LAYER
!chalice new-project $PROJECT_LAYER

In [ ]:
!tree -a $PROJECT_LAYER/.

In [ ]:
%%writefile $PROJECT_LAYER/.chalice/config.json
{
  "version": "2.0",
  "app_name": "layerapp",
  "automatic_layer": true,
  "stages": {
    "dev": {
      "api_gateway_stage": "api"
    }
  }
}

In [ ]:
%%writefile $PROJECT_LAYER/app.py

from chalice import Chalice
app = Chalice(app_name='layerapp')

@app.lambda_function()
def boto3_import(event, context):
    import boto3
    user_name = event.get('user_name')
    return {'boto3': boto3.__file__, 'user_name': user_name}

@app.lambda_function()
def pandas_import(event, context):
    import pandas
    user_name = event.get('user_name')    
    return {'pandas': pandas.__file__, 'user_name': user_name}

In [ ]:
%%writefile $PROJECT_LAYER/requirements.txt
boto3==1.26.91
pandas==1.2.4

In [ ]:
!cd $PROJECT_LAYER && chalice deploy

AWS 콘솔에서 배포된 람다 함수를 확인합니다.

In [ ]:
from IPython.core.display import display, HTML
import boto3
region = boto3.Session().region_name
def make_lambda_console_link(region, func_name):
    link = f'<b><a target="blank" href="https://{region}.console.aws.amazon.com/lambda/home?region={region}#/functions/{func_name}?tab=code">Check Lambda function: {func_name}</a></b>'   
    return link  

func_name = f"{PROJECT_LAYER}-dev-boto3_import"        
link = make_lambda_console_link(region, func_name)
display(HTML(link))

func_name = f"{PROJECT_LAYER}-dev-pandas_import"        
link = make_lambda_console_link(region, func_name)
display(HTML(link))

invoke 커맨드로 람다 함수를 테스트합니다.

In [ ]:
%%bash -s "$PROJECT_LAYER"
cd $1

echo '{"user_name": "daekeun"}' | chalice invoke -n boto3_import
echo '{"user_name": "gildong"}' | chalice invoke -n boto3_import

echo '{"user_name": "daekeun"}' | chalice invoke -n pandas_import
echo '{"user_name": "gildong"}' | chalice invoke -n pandas_import

## Reuse Layer

람다 레이어를 재사용하려면 `.chalice/config.json` 파일에 레이어 구성 옵션을 추가합니다. (`'layers': [YOUR-LAYER-ARN`)

In [ ]:
PROJECT_REUSE = "reuse-layer"
!rm -rf $PROJECT_REUSE
!chalice new-project $PROJECT_REUSE

In [ ]:
client = boto3.client('lambda')
response = client.get_function(
    FunctionName='layerapp-dev-boto3_import',
)
layer_arn = response['Configuration']['Layers'][0]['Arn']
print(layer_arn)

위 셀의 결과를 붙여넣기하거나 앞 섹션에서 chalice deploy 커맨드의 출력값 중 lambda layer arn을 붙여넣기합니다.

```
Resources deployed:
  - Lambda Layer ARN: arn:aws:lambda:us-east-1:123456789123:layer:layerapp-dev-managed-layer:1
  - Lambda ARN: arn:aws:lambda:us-east-1:123456789123:function:layerapp-dev-boto3_import
  - Lambda ARN: arn:aws:lambda:us-east-1:123456789123:function:layerapp-dev-pandas_import
```

In [ ]:
%%writefile $PROJECT_REUSE/.chalice/config.json
{
  "version": "2.0",
  "app_name": "reuse-layer",
  "layers": ["YOUR-LAYER-ARN"],
  "stages": {
    "dev": {
      "api_gateway_stage": "api"
    }
  }
}

In [ ]:
%%writefile $PROJECT_REUSE/app.py

from chalice import Chalice
app = Chalice(app_name='reuse-layer')

@app.lambda_function()
def pandas_import_reuse(event, context):
    import pandas
    user_name = event.get('user_name')
    return {'pandas': pandas.__file__, 'user_name': user_name}

In [ ]:
!cd $PROJECT_REUSE && chalice deploy

In [ ]:
%%bash -s "$PROJECT_REUSE"
cd $1

echo '{"user_name": "daekeun"}' | chalice invoke -n pandas_import_reuse
echo '{"user_name": "gildong"}' | chalice invoke -n pandas_import_reuse


In [ ]:
!du -hs $PROJECT_LAYER/.chalice/deployments/*
!du -hs $PROJECT_REUSE/.chalice/deployments/*

In [ ]:
!cd $PROJECT_LAYER && chalice delete
!cd $PROJECT_REUSE && chalice delete